In [305]:
# %load ~/.ipython/standard_imports.py
import os
import sys
import logging
import itertools

logging.basicConfig(level=logging.INFO)
import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.preprocessing

import isajosep_util
import isajosep_util.data_frame_plotter

In [5]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


Attached CSV file contains a number of medical cases (extracted from EHRs). Each entry consists of:

- one more more “present” symptoms (symptoms that patient had at the time of visit). For example `s_0136` is GYANT code for “earache”.
- one more “absent” symptoms (symptoms that patient did not have). Keep in mind that there may other potential symptoms the patient was never asked about, which are neither “present” nor “absent”
- Age, Sex (1=Male), and Month of visit (which may be helpful, e.g. some conditions are gender-specific, some are seasonal)
- Diagnosis (“DX”) the patient was diagnosed with (using Gyant condition codes)


# Load

In [10]:
df = pd.read_csv('/Users/ijoseph/Code/Data/Gyant/data_challenge.csv', index_col=0)

# Cleaning

In [12]:
df.sample(5)

,DX,AGE,MONTH,SEX,Absent,Present
59908,c_0497,37.0,8,0,"s_0242, s_0865, s_0346, s_0327","s_1324, s_1547, s_0496, s_1316, s_0847"
8497,c_0133,19.0,8,0,NaN,"s_0242, s_0824, s_1266, s_2697"
62265,c_0608,63.0,1,1,"s_0553, s_1213, s_1611, s_0242, s_2738, s_0084...","s_0445, s_2282"
66587,c_0273,36.0,7,0,"s_0070, s_0553, s_0078, s_0084, s_2563, s_0022...",NaN
27272,c_0145,4.0,2,0,s_0106,"s_0136, s_0242, s_0309, s_0826, s_0180"


In [13]:
df.shape

(71662, 6)

In [14]:
df.describe()

,AGE,MONTH,SEX
count,71662.000000,71662.000000,71662.000000
mean,32.297787,6.197357,0.407538
std,20.104694,3.531117,0.491380
min,-3.000000,1.000000,0.000000
25%,19.000000,3.000000,0.000000
50%,32.000000,6.000000,0.000000
75%,45.000000,9.000000,1.000000
max,161.000000,12.000000,1.000000


Hm, mostly reasonable except for some impossible values for `AGE`.

## `AGE` cleaning

In [16]:
df.query("AGE > 100")

,DX,AGE,MONTH,SEX,Absent,Present
1186,c_0273,102.0,8,0,"s_0070, s_0106, s_0039, s_0346, s_0519, s_0002...","s_0578, s_0120, s_0865, s_2563"
24878,c_0497,159.0,12,1,s_1298,"s_0210, s_1317, s_2194, s_2387, s_1316"
24893,c_0036,101.0,7,0,"s_0070, s_1047, s_0519, s_0078, s_0022, s_0062",NaN
44092,c_0720,161.0,1,0,s_0327,s_1216
47754,c_0608,102.0,4,0,"s_0400, s_0553, s_0826, s_0180","s_0022, s_2253, s_1030, s_0901, s_0460"
57856,c_0273,102.0,1,0,"s_0070, s_0864, s_0542, s_0039, s_0078","s_0578, s_0647, s_2738, s_1266"


In [18]:
df.query("AGE < 0")

,DX,AGE,MONTH,SEX,Absent,Present
14105,c_0273,-3.0,8,0,"s_2738, s_2282","s_2204, s_0578, s_0542, s_2563, s_1266"


Okay anything over 102.0 seems unlikely, and < 0 impossible. Want to assess whether missingness is correlated with anything else, rather than merely dropping these off the bat so as to not induce bias from dropping. 

In [20]:
df.query("(AGE > 103) or (AGE < 0) ")

,DX,AGE,MONTH,SEX,Absent,Present
14105,c_0273,-3.0,8,0,"s_2738, s_2282","s_2204, s_0578, s_0542, s_2563, s_1266"
24878,c_0497,159.0,12,1,s_1298,"s_0210, s_1317, s_2194, s_2387, s_1316"
44092,c_0720,161.0,1,0,s_0327,s_1216


Nothing obvious in terms of presence /absence of symptoms or DX, so safe to drop these, probably. 

In [21]:
df_age_cleaned = df.query("(AGE < 103) and (AGE > 0)")

## Missing/ Duplicated Values

Using custom helper function to check for both ([see repository here](https://github.com/ijoseph/util/blob/6d6cd84da663c39009d3894977f3e61c88d0969b/isajosep_util/__init__.py#L20))

In [23]:
isajosep_util.check_for_null(df_age_cleaned)

✗ 12,320 NaN or Inf values; 0 inf, 12,320 nan; Duplications: 185 duplicated rows by exclusively non-index content, 0 duplicated rows by exclusively index, 0 duplicated by both


(12320, 0, 12320, 185, 0, 0)

Okay, so several NaNs as expected via lack of check for absence or presence. 

Duplications are less expected. Looking more into these... 

### Duplicated by all columns other than index

In [36]:
df_age_cleaned[df_age_cleaned.duplicated(keep=False)].sort_values(by=['DX','AGE', 'MONTH', 'SEX']).head(10)

,DX,AGE,MONTH,SEX,Absent,Present
33339,c_0008,29.0,12,0,NaN,NaN
37010,c_0008,29.0,12,0,NaN,NaN
14403,c_0008,30.0,3,0,NaN,NaN
19340,c_0008,30.0,3,0,NaN,NaN
960,c_0008,30.0,6,1,NaN,NaN
59748,c_0008,30.0,6,1,NaN,NaN
29459,c_0008,31.0,1,1,NaN,NaN
53960,c_0008,31.0,1,1,NaN,NaN
32905,c_0008,31.0,6,0,NaN,NaN
49801,c_0008,31.0,6,0,NaN,NaN


In [39]:
df_age_cleaned_dedup = df_age_cleaned.drop_duplicates(subset=['DX', 'AGE', 'MONTH', 'Absent', 'Present'])

In [41]:
df_age_cleaned_dedup.shape, df_age_cleaned.shape

((69305, 6), (69646, 6))

### Duplicated by `DX`, `AGE`, `MONTH`, `SEX`

In [74]:
dup_subset = ['DX', 'AGE', 'MONTH', 'SEX']

In [75]:
isajosep_util.check_for_null(df_age_cleaned_dedup.drop(['Absent', 'Present'], axis='columns'))

✗ 0 NaN or Inf values; 0 inf, 0 nan; Duplications: 48,083 duplicated rows by exclusively non-index content, 0 duplicated rows by exclusively index, 0 duplicated by both


(0, 0, 0, 48083, 0, 0)

In [65]:
print("{:,} of {:,} rows are part of a duplication event!".format(df_age_cleaned_dedup.drop(['Absent', 'Present'], axis='columns').duplicated(keep=False).sum(), df_age_cleaned_dedup.shape[0]))

60,578 of 69,305 rows are part of a duplication event!


Okay, interesting in that lots of duplications when we get rid of those columns for some reason. Off the bat, seems like too many duplications to be by chance; how many people with the exact same age (to the month) and gender could there be receiving the same diagnosis? 

Therefore, we need to merge the `Absent` and `Present` columns for rows with duplicate values of `( DX, AGE, MONTH, SEX)`. The assumption here is that duplicate diagnoses are simply multiple entires of the same real-world diagnostic test, rather than duplicate relatively-independent real-world tests having been performed.

In [76]:
df_age_cleaned_dedup[df_age_cleaned_dedup.duplicated(subset=dup_subset, keep=False)].sort_values(dup_subset).head()

,DX,AGE,MONTH,SEX,Absent,Present
28287,c_0008,1.0,1,1,"s_0553, s_0180","s_0084, s_0443"
40964,c_0008,1.0,1,1,"s_0106, s_0542, s_0553, s_0363, s_0180",s_0084
1035,c_0008,2.0,5,1,"s_0434, s_0309, s_1141, s_0084, s_0180","s_0400, s_0445, s_0022, s_0826"
44996,c_0008,2.0,5,1,"s_0911, s_0180, s_0268, s_1141, s_0542, s_0826...",s_0084
52058,c_0008,2.0,6,1,NaN,"s_2194, s_0309, s_0826"


Can probalby fix this with [`groupby` and an `aggregate` function](https://stackoverflow.com/questions/36271413/pandas-merge-nearly-duplicate-rows-based-on-column-value). 

#### `aggregate` function

In [166]:
def combine_symptom_cells(cell_list):
    """
    Takes in a list of cells, each of which are comma-delim strings (or just NaN which need to be ignored). 
    
    Outputs a de-duplicated tuple of combined symptom codes. 
    
    Example: 
    ['s_0553, s_0180', np.nan,'s_0106, s_0542, s_0553, s_0363, s_0180'] -> 
    ('s_0363', 's_0106', 's_0180', 's_0542', 's_0553')    
    """
    
    # Parse from string to list of string by commas
    parsed = [ c.split(',') for c in cell_list if isinstance(c,str) ]
    
    # Flatten 
    parsed = [item for sublist in parsed for item in sublist]
    
    #Strip whitespace
    parsed = [s.strip() for s in parsed]
    
    # Remove duplciates
    parsed = tuple(set(parsed))
    
    return(parsed)    

##### Test

In [167]:
tst_arr = ['s_0553, s_0180', np.nan,'s_0106, s_0542, s_0553, s_0363, s_0180']

In [168]:
combine_symptom_cells(tst_arr)

('s_0363', 's_0106', 's_0180', 's_0542', 's_0553')

##### Use

In [169]:
df_aggregated = df_age_cleaned_dedup.groupby(dup_subset).aggregate(combine_symptom_cells).reset_index()

In [170]:
df_aggregated.shape

(21222, 6)

In [172]:
df_aggregated.sample(10)

,DX,AGE,MONTH,SEX,Absent,Present
636,c_0008,41.0,2,1,"(s_0620, s_1211, s_0553, s_0605)","(s_2282, s_0511, s_0039)"
14102,c_0273,45.0,9,0,"(s_2243, s_1547, s_0434, s_0062, s_0327, s_010...","(s_0798, s_1005, s_1302, s_0300, s_0864, s_024..."
2275,c_0036,50.0,7,0,"(s_0400, s_0136, s_0363, s_0085, s_0242, s_000...","(s_2734, s_0084, s_0180, s_0022, s_0553, s_082..."
14238,c_0273,51.0,11,1,"(s_0363, s_0084, s_0002, s_0180, s_1030, s_082...","(s_0634, s_2734, s_2194, s_0078, s_2282, s_002..."
13852,c_0273,34.0,12,0,"(s_0478, s_0180, s_0022, s_0118, s_0120)","(s_0078, s_0070)"
11675,c_0196,6.0,11,1,"(s_0363, s_0864, s_0002, s_0084, s_0106, s_018...","(s_0400, s_0434, s_2194, s_0084, s_0180, s_054..."
16092,c_0497,75.0,8,0,"(s_2243, s_0124, s_0039, s_0242, s_0002, s_008...","(s_1547, s_0270, s_2367, s_1316, s_2252, s_060..."
8244,c_0152,44.0,10,0,"(s_0506, s_0136, s_1611, s_0362, s_0085, s_086...","(s_0136, s_0400, s_0478, s_0084, s_0180, s_118..."
2830,c_0036,76.0,9,0,"(s_1547, s_0124, s_0084, s_0002, s_0180, s_002...","(s_2734, s_0786, s_0227, s_0084, s_0078, s_105..."
11452,c_0187,67.0,2,1,"(s_0837, s_0180)","(s_0826, s_0242)"


In [171]:
isajosep_util.check_for_null(df_aggregated)

✓ No Nan or Inf values; no duplications.


(0, 0, 0)

# Formatting

## Index by `[DX, AGE, MONTH, SEX]`.

In [173]:
df_aggregated.set_index(['DX', 'AGE', 'MONTH', 'SEX'], inplace=True)

In [174]:
df_aggregated.sample(3)

,,,,Absent,Present
DX,AGE,MONTH,SEX,,
c_0497,26.0,9,1,"(s_0124, s_0002, s_0084, s_0180, s_0022, s_024...","(s_1759, s_1547, s_0270, s_1316, s_0242, s_219..."
c_0187,1.0,9,0,"(s_0542, s_0180, s_0553, s_2742)","(s_0400, s_0084, s_0309, s_1155, s_0826)"
c_0273,35.0,2,1,"(s_1611, s_0124, s_0434, s_0106, s_0180, s_054...","(s_2734, s_0242, s_0002, s_0106, s_1266, s_154..."


## Create flags for each symptom checked so as to be useful as features for ML modeling.  

Basic idea: for each patient, each symptom can have one of four states, each of which might have some diagnostic utility: 
1. Symptom not checked for* (`not_checked`)
2. Symptom checked for and present. (`checked_and_present`)
3. Symptom checked for and absent. (`checked_and_absent`)
4. Symptom checked for and (present AND absent)† (`checked_and_present_and_absent`) 

†The fourth option is contradictory at first glance, but potentially possible if symptoms checked at, for example, different times within a month and give different results on each check. 

*Option one is of interested because if symptom wasn't checked for, that might be of particular diagnostic value (perhaps the patient was unable to consent to the diagnostic, which is useful unformation in and of itself in terms of the pateint's verbal status, for example). 

Concrete encoding: 
For each symptom `A`, `sklearn` will convert to boolean indicators with the folowing meaning:


|Indicator   |  Meaning |
|---|---|
|`absentA`   |  $0$: Symptom not found in `Absent` column; $1$: symptom found in `Absent` column  |
|`presentA`   |  $0$: Symptom not found in `Present` column; $1$: sympton found in `Present` column  |


We will convert this to the four options above for interpretability: 

| `absentA` | `presentA` | *state*|
|---|---|---|
| 0| 0 | `not_checked` |
| 0 |1 | `checked_and_present` |
|1  | 0 | `checked_and_absent` |
| 1 | 1 | `checked_and_present_and_absent`|

In [225]:
smpl = df_aggregated.sample(2)

In [226]:
smpl

,,,,Absent,Present
DX,AGE,MONTH,SEX,,
c_0196,43.0,4,0,"(s_0400, s_0363, s_0242, s_0084, s_0180, s_054...","(s_0400, s_0136, s_0478, s_0864, s_0084, s_018..."
c_0497,14.0,1,1,"(s_0136, s_0084, s_0002, s_1266, s_0542, s_018...","(s_0496,)"


In [233]:
binarizer = sklearn.preprocessing.MultiLabelBinarizer()
binarizer = binarizer.fit(smpl.Absent + smpl.Present) # fit on both present and absent classes

In [236]:
absent_df = pd.DataFrame(binarizer.transform(smpl.Absent), columns=binarizer.classes_, index=smpl.index)

In [237]:
absent_df

,,,,s_0002,s_0022,s_0084,s_0136,s_0180,s_0242,s_0278,s_0346,s_0363,s_0400,s_0445,s_0478,s_0496,s_0542,s_0553,s_0701,s_0826,s_0837,s_0864,s_1266,s_1542,s_2282,s_2739
DX,AGE,MONTH,SEX,,,,,,,,,,,,,,,,,,,,,,,
c_0196,43.0,4,0,0,0,1,0,1,1,0,0,1,1,1,0,0,1,1,1,1,1,0,1,1,0,0
c_0497,14.0,1,1,1,1,1,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0


`absent_df`: each column indicates whether sympton explicitly absent. 

In [239]:
present_df = pd.DataFrame(binarizer.transform(smpl.Present), columns=binarizer.classes_, index=smpl.index)

In [240]:
present_df

,,,,s_0002,s_0022,s_0084,s_0136,s_0180,s_0242,s_0278,s_0346,s_0363,s_0400,s_0445,s_0478,s_0496,s_0542,s_0553,s_0701,s_0826,s_0837,s_0864,s_1266,s_1542,s_2282,s_2739
DX,AGE,MONTH,SEX,,,,,,,,,,,,,,,,,,,,,,,
c_0196,43.0,4,0,0,0,1,1,1,0,1,1,0,1,1,1,0,0,0,0,0,0,1,0,0,1,1
c_0497,14.0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


`present_df`: each column indicates whether symptom explicitly present.

In [270]:
pres_and_abs_df = present_df.join(absent_df, lsuffix="__present", rsuffix="__absent")

### Iterate through and create new DataFrame

For each column, loook up and create new vector (`not_checked`, `checked_and_present`, `checked_and_absent`, `checked_and_present_and_absent`) 

In [271]:
possibilities = ['not_checked', 'checked_and_present', 'checked_and_absent', 'checked_and_present_and_absent']

In [272]:
all_possible_symptoms_and_possibilites = ["{}__{}".format(symptom, status)  for (symptom,  status) in itertools.product(binarizer.classes_, possibilities)]

Create placeholder

In [273]:
indicator_df = pd.DataFrame(columns=all_possible_symptoms_and_possibilites, index=smpl.index)

In [274]:
tqdm.tqdm_pandas(tqdm.tqdm_notebook())

| `absentA` | `presentA` | *state*|
|---|---|---|
| 0| 0 | `not_checked` |
| 0 |1 | `checked_and_present` |
|1  | 0 | `checked_and_absent` |
| 1 | 1 | `checked_and_present_and_absent`|

In [314]:
def convert(row, symptoms):
    """
    Convert a row of a DataFrame containing colums with names `symtom`_present, `symptom`_absent for symptom in symptoms
    to a row with ['`symtom`__not_checked', `symptom`__'checked_and_present', `symtom`__'checked_and_absent', `symtom__'checked_and_present_and_absent']. 
    
    e.g. 
    
    s_0002__present    0
    s_0002__absent     1
    
    -> 

    s_0002__not_checked  0
    s_0002__checked_and_present 0 
    s_0002__checked_and_absent 1 
    s_0002__checked_and_present_and_absent 0
    
    """
    
    result_series = pd.Series(data=[0]*len(all_possible_symptoms_and_possibilites), index=all_possible_symptoms_and_possibilites)        
    
    for symptom in symptoms:
        try:
            absent = row[symptom + "__absent"]
            present = row[symptom + "__present"]
        except KeyError:
            sys.stderr.write("Warning: symptom {} not found".format(symptom))
            continue

        if absent == 0 and present == 0:
            result_series["{}__not_checked".format(symptom)] = 1
        elif absent == 0 and present == 1:
            result_series["{}__checked_and_present".format(symptom)] = 1
        elif absent == 1 and present == 0:
            result_series["{}__checked_and_absent".format(symptom)] = 1
        elif absent == 1 and present == 1:
            result_series["{}__checked_and_present_and_absent".format(symptom)] = 1
        else:
            raise ValueError("Some issue with encoding; symptom = {}".format(symptom))    
    return result_series

#### Test

In [315]:
pres_and_abs_df.filter(like='s_0002').iloc[1,:]

s_0002__present    0
s_0002__absent     1
Name: (c_0497, 14.0, 1, 1), dtype: int64

In [317]:
binarizer.classes_[]

array(['s_0002', 's_0022', 's_0084', 's_0136', 's_0180', 's_0242',
       's_0278', 's_0346', 's_0363', 's_0400', 's_0445', 's_0478',
       's_0496', 's_0542', 's_0553', 's_0701', 's_0826', 's_0837',
       's_0864', 's_1266', 's_1542', 's_2282', 's_2739'], dtype=object)

In [319]:
convert(pres_and_abs_df.filter(like='s_0002').iloc[1,:], symptoms=['s_0002'])

s_0002__not_checked                       0
s_0002__checked_and_present               0
s_0002__checked_and_absent                1
s_0002__checked_and_present_and_absent    0
s_0022__not_checked                       0
s_0022__checked_and_present               0
s_0022__checked_and_absent                0
s_0022__checked_and_present_and_absent    0
s_0084__not_checked                       0
s_0084__checked_and_present               0
s_0084__checked_and_absent                0
s_0084__checked_and_present_and_absent    0
s_0136__not_checked                       0
s_0136__checked_and_present               0
s_0136__checked_and_absent                0
s_0136__checked_and_present_and_absent    0
s_0180__not_checked                       0
s_0180__checked_and_present               0
s_0180__checked_and_absent                0
s_0180__checked_and_present_and_absent    0
s_0242__not_checked                       0
s_0242__checked_and_present               0
s_0242__checked_and_absent      